In [1]:
import torch
import pickle

In [2]:
seed = 0

In [3]:
dataset_a = [[6, 8, 7, 4, 2, 3, 0],
             [1, 8, 7, 4, 2, 3, 5],
             [6, 3, 4, 2, 7, 8, 5],
             [1, 3, 4, 2, 7, 8, 0],
             [0, 9, 7, 8, 5, 3, 4, 1],
             [2, 9, 7, 8, 5, 3, 4, 6],
             [0, 4, 3, 5, 8, 7, 9, 6],
             [2, 4, 3, 5, 8, 7, 9, 1]]

dataset_b = [[6, 8, 7, 4, 2, 3, 5],
             [1, 8, 7, 4, 2, 3, 0],
             [6, 3, 4, 2, 7, 8, 0],
             [1, 3, 4, 2, 7, 8, 5],
             [0, 9, 7, 8, 5, 3, 4, 6],
             [2, 9, 7, 8, 5, 3, 4, 1],
             [0, 4, 3, 5, 8, 7, 9, 1],
             [2, 4, 3, 5, 8, 7, 9, 6]]

In [4]:
class Encoder():
    def __init__(self):
        self.seen = set()

    def noise(self):
        n = int(torch.randint(50000, (1, )) + 10)
        while n in self.seen:
            n = int(torch.randint(50000, (1, )) + 10)
        self.seen.add(n)
        return int(n)

In [5]:
def refresh_clean(sequence, target, it):
    dataset = dataset_a if it < 10000 else dataset_b
    torch.manual_seed(seed+it)
    c = list(dataset[torch.randint(0, len(dataset), (1, ))]) + [-1]
    encoder.noise()
    t = c[1:] + [-1]
    sequence.extend(c)
    target.extend(t)
    
def refresh_noisy(sequence, target, it):
    dataset = dataset_a if it < 10000 else dataset_b
    torch.manual_seed(seed+it)
    c = list(dataset[torch.randint(0, len(dataset), (1, ))]) + [encoder.noise()]
    t = c[1:] + [-1]
    sequence.extend(c)
    target.extend(t)

In [6]:
encoder = Encoder()
def generate_stream(refresh):
    torch.manual_seed(seed)
    history, sequence, target = [], [], []
    refresh(sequence, target, 0)
    for it in range(0, 20000):
        csymbol = sequence.pop(0)
        tsymbol = target.pop(0)
        history.append(csymbol)
        if len(sequence) == 0:
            refresh(sequence, target, it)
    return history + sequence

In [7]:
clean_stream = generate_stream(refresh_clean)
noisy_stream = generate_stream(refresh_noisy)

In [8]:
data = {'clean': clean_stream, 'noisy': noisy_stream}

In [9]:
file = open('dataset.pkl', 'wb')
pickle.dump(data, file)